In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.metrics import ndcg_score
import utils
import polars as pl

# compute metrics for all datasets and models

In [29]:
def compute_metrics(model_folder, dataset, relevance='binary_strict'):
    r = pd.read_parquet(f'{model_folder}/{dataset}-ranking_with_relevance.parquet')

    if relevance=='binary_strict':
        if dataset == 'wands':
            r.relevance=[1 if i==2 else 0 for i in r.relevance]
        elif dataset.startswith('esci'):
            r.relevance=[1 if i==4 else 0 for i in r.relevance]

    elif relevance=='binary_tolerant':
        if dataset == 'wands':
            r.relevance=[1 if i>0 else 0 for i in r.relevance]
        elif dataset.startswith('esci'):
            r.relevance=[1 if i>1 else 0 for i in r.relevance]
    
    elif relevance!='asis':
        raise ValueError(f"invalid relevance string {relevance}'")
    
    df_dcg, ndcg = utils.compute_ndcg(r)

    custom_ndcg = ndcg.ndcg.mean()

    sklearn_ndcg = []
    for qid in tqdm(np.unique(r.query_id)):
        rq = r[r.query_id==qid]
        sklearn_ndcg.append(ndcg_score(rq.relevance.values.reshape(1,-1), 1/rq.model_rank.values.reshape(1,-1)))

    sklearn_ndcg = np.mean(sklearn_ndcg)

    return {'custom_ndcg': custom_ndcg, 'sklearn_ndcg': sklearn_ndcg}

## relevance strict

In [ ]:
r = {}

for model_folder in ['openai', 'text-exp0307', 'text-004']:
    r[model_folder] = {}
    for dataset in ['wands', 'esci-es', 'esci-us']:
        print(model_folder, dataset, flush=True)
        r[model_folder][dataset] = compute_metrics(model_folder, dataset)

openai wands


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1205.37it/s]

openai esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:14<00:00, 1069.47it/s]


openai esci-us


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1544.17it/s]

text-exp0307 esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:14<00:00, 1079.02it/s]

text-exp0307 esci-us



 44%|██████████████████████████████████████████████████████████████████▋                                                                                    | 43016/97345 [03:04<04:04, 221.77it/s]

In [34]:
1

1

In [36]:
pd.DataFrame({m: {d: r[m][d]['custom_ndcg'] for d in r[m].keys()} for m in r.keys()}).T

,wands,esci-es,esci-us
openai,0.825493,0.724324,0.688415
text-exp0307,0.858814,0.736770,0.705098
text-004,0.838700,0.653741,0.671455


In [37]:
pd.DataFrame({m: {d: r[m][d]['sklearn_ndcg'] for d in r[m].keys()} for m in r.keys()}).T

,wands,esci-es,esci-us
openai,0.615339,0.676370,0.623495
text-exp0307,0.637729,0.696486,0.650716
text-004,0.618541,0.554086,0.605527


# relevance tolerant

In [42]:
r = {}

for model_folder in ['openai', 'text-exp0307', 'text-004']:
    r[model_folder] = {}
    for dataset in ['wands', 'esci-es', 'esci-us']:
        print(model_folder, dataset, flush=True)
        r[model_folder][dataset] = compute_metrics(model_folder, dataset, relevance='binary_tolerant')

openai wands


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1573.58it/s]

openai esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:13<00:00, 1085.15it/s]

openai esci-us


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1547.26it/s]

text-exp0307 esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:13<00:00, 1085.95it/s]

text-exp0307 esci-us



 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                              | 78057/97345 [05:43<01:29, 215.63it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97345/97345 [05:05<00:00, 318.40it/s]


In [48]:
pd.DataFrame({m: {d: r[m][d]['custom_ndcg'] for d in r[m].keys()} for m in r.keys()})

,openai,text-exp0307,text-004
wands,0.926686,0.930909,0.916136
esci-es,0.738400,0.751340,0.666371


In [49]:
pd.DataFrame({m: {d: r[m][d]['sklearn_ndcg'] for d in r[m].keys()} for m in r.keys()})

,openai,text-exp0307,text-004
wands,0.901472,0.901683,0.893235
esci-es,0.705468,0.723771,0.586476


# relevance as is

In [45]:
r = {}

for model_folder in ['openai', 'text-exp0307', 'text-004']:
    r[model_folder] = {}
    for dataset in ['wands', 'esci-es']:
        print(model_folder, dataset, flush=True)
        r[model_folder][dataset] = compute_metrics(model_folder, dataset, relevance='asis')

openai wands


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1460.75it/s]

openai esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:14<00:00, 1053.79it/s]

text-exp0307 wands


...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:14<00:00, 1057.72it/s]

text-004 wands



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1511.92it/s]

text-004 esci-es



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15180/15180 [00:14<00:00, 1053.68it/s]


In [46]:
pd.DataFrame({m: {d: r[m][d]['custom_ndcg'] for d in r[m].keys()} for m in r.keys()})

,openai,text-exp0307,text-004
wands,0.926686,0.930909,0.916136
esci-es,0.738400,0.751340,0.666371


In [50]:
pd.DataFrame({m: {d: r[m][d]['sklearn_ndcg'] for d in r[m].keys()} for m in r.keys()})

,openai,text-exp0307,text-004
wands,0.901472,0.901683,0.893235
esci-es,0.705468,0.723771,0.586476


# summarize datasets

In [39]:
import polars as pl

datasets = ['wands', 'esci-es', 'esci-us']

stats = []

for dataset in datasets:
    print (dataset, flush=True)
    p = pl.read_parquet(f'{dataset}/products.parquet').to_pandas()
    q = pd.read_parquet(f'{dataset}/queries.parquet')
    r = pd.read_parquet(f'{dataset}/relevance.parquet')

    s = {'num_products': len(p), 
         'num_queries': len(q), 
         'num_relevance_judgements': len(r),
         'num_queries_with_relevance': len(np.unique(r.query_id)),
         'num_products_with_relevance': len(np.unique(r.product_id)),
         'relevance_values_distribution': r.relevance.value_counts().sort_index(ascending=False) / len(r),
         'mean_num_relevance_judgements_per_query': r.groupby('query_id').size().values.mean()
        }
    stats.append(s)

stats = pd.DataFrame(stats, index=datasets).T

wands
esci-es
esci-us


In [40]:
stats

,wands,esci-es,esci-us
num_products,42994,259973,1215851
num_queries,480,15180,97345
num_relevance_judgements,233448,356410,1818825
num_queries_with_relevance,480,15180,97345
num_products_with_relevance,42986,259973,1215851
relevance_values_distribution,relevance 2 0.109720 1 0.628118 0 0.2...,relevance 4 0.566948 3 0.057526 2 0.2...,relevance 4 0.685914 3 0.022019 2 0.2...
mean_num_relevance_judgements_per_query,486.35,23.47892,18.684319


In [41]:
for dataset in datasets:
    print (stats.loc['relevance_values_distribution'][dataset])

relevance
2    0.109720
1    0.628118
0    0.262161
Name: count, dtype: float64
relevance
4    0.566948
3    0.057526
2    0.249878
1    0.125647
Name: count, dtype: float64
relevance
4    0.685914
3    0.022019
2    0.203050
1    0.089016
Name: count, dtype: float64


# manual calculation for inspection

In [5]:
#model_folder = 'openai'
#model_folder = 'text-exp0307'
model_folder = 'text-004'
#dataset = 'wands'
dataset = 'esci-us'

r = pd.read_parquet(f'{model_folder}/{dataset}-ranking_with_relevance.parquet')
r.head()    

,query_id,product_id,model_rank,relevance
0,0,B074RDGKH4,1,0.0
1,0,B008OHPUYM,2,0.0
2,0,B004KLWPI0,3,0.0
3,0,B0057GS2WK,4,0.0
4,0,B07K1X8CT7,5,0.0


In [6]:
len(np.unique(r.query_id)), len(np.unique(r.product_id))

(97345, 1045194)

In [7]:
p = pl.read_parquet(f'{dataset}/products.parquet').to_pandas()
p.set_index('__index_level_0__', inplace=True)
p.head()

,product
__index_level_0__,
0000306045,<TITLE>WhiteCoat Clipboard- Pink - Respiratory...
0002159511,<TITLE>A Day in the Life of Ireland</TITLE>\n<...
0003002764,<TITLE>HENRY'S AWFUL MISTAKE</TITLE>
0004126475,<TITLE>David Bellamy's Watercolour Landscape C...
0004127579,<TITLE>Developing Your Watercolours</TITLE>\n<...


In [8]:
p.shape

(1215851, 1)

In [9]:
if dataset == 'wands':
    r.relevance=[1 if i==2 else 0 for i in r.relevance]
elif dataset.startswith('esci'):
    r.relevance=[1 if i==4 else 0 for i in r.relevance]

In [10]:
r.relevance.value_counts()

relevance
0    3266336
1     627464
Name: count, dtype: int64

## custom ndcg

In [11]:
df_dcg, ndcg = utils.compute_ndcg(r)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97345/97345 [07:13<00:00, 224.62it/s]


In [12]:
ndcg.ndcg.mean()

np.float64(0.671454768707987)

## sklearn ndcg

In [13]:
sklearn_ndcg = []
for qid in tqdm(np.unique(r.query_id)):
    rq = r[r.query_id==qid]
    sklearn_ndcg.append(ndcg_score(rq.relevance.values.reshape(1,-1), 1/rq.model_rank.values.reshape(1,-1)))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97345/97345 [05:09<00:00, 314.69it/s]


In [15]:
np.mean(sklearn_ndcg)

np.float64(0.6055267325550163)

In [62]:
ndcg.ndcg.mean()

np.float64(0.7243239003217897)

# custom ndcg

```
                 wands           esci-es          esci-es        esci-us
               relevance-01   relevance-01234   relevance-01    relevance-01
text-004          0.839           0.666            0.654          0.671
text-ext307       0.859           0.751            0.751          0.705          
openai            0.825           0.739            0.724
```

## skelarn ndcg

```
                 wands           esci-es          esci-es
               relevance-01   relevance-01234   relevance-01
text-004          0.619           0.586              0.554         0.606       
text-ext307       0.638           0.724              0.696
openai            0.615           0.705              0.676
```

In [76]:
pd.DataFrame({m: {d: r[m][d]['custom_ndcg'] for d in r[m].keys()} for m in r.keys()}).T

,wands,esci-es
openai,0.825493,0.724324
text-exp0307,0.858814,0.736770
text-004,0.838700,0.653741
